# Amazon Bedrock에서 Claude 3.5 Sonnet 모델을 사용해 주어진 토픽에 대한 파워포인트 슬라이드를 만들어보기

### 환경 설정

#### 참조:  Bedrock 호출
- 참고: https://docs.anthropic.com/en/api/claude-on-amazon-bedrock


In [1]:
# pip install boto3

In [2]:
%load_ext autoreload
%autoreload 2

## 1: Chaininig Prompt를 사용해 파워포인트 슬라이드 생성하기

- 목표: 최소한의 API 호출로 Chaining Prompt를 사용하여 20장의 파워포인트 슬라이드를 생성하기
- 아이디어: 파워포인트 슬라이드 20장을 구성하기 위해 3번의 Bedrock invoke_model API를 호출한다. 
    - (1) Initial prompt를 사용해 전체 아웃라인 짜기 - 즉, 1번째 Bedrock invoke_model API를 호출에서는 토픽에 맞는 전체 개요를 만든다.
    - (2) Slide prompt를 사용해 1번에서 짠 아웃라인에 맞춰서 디테일한 내용을 불렛 포인트로 정리한 슬라이드를 각각 만든다 - 즉, 2번째 호출에서는 1~10번째 슬라이드의 개요를, 3번째 호출에서는 11~20번째 슬라이드 개요만을 사용해 각 슬라이드에 대한 내용을 자세하게 생성한다.

### 템플릿 파일 기술

In [3]:
from util.cross_region_inference_prompt_util import *

# 템플릿 파일의 경로를 지정
prompt_folder = "prompt_template"
outline_prompt_file_name = "outline_prompt_template.txt"

### Outline prompt template 로딩 및 토픽 추가

In [4]:
from jinja2 import Template

prompt_file_path = os.path.join(prompt_folder, outline_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Closing Ceremony Success Prediction"
outline_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)

print(outline_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Analyze the given topic thoroughly to write in the provided <output_format> JSON format.
Return only the JSON object without additional tags or explanations.

<context>
Paris 2024 Olympics Closing Ceremony Success Prediction
</context>

<instruction>
1. Title:
   - Create a compelling and concise main title that captures the essence of the presentation.

2. Slides:
   - Generate 20 slides, including an opening and a closing slide.
   - For each slide:
     - Assign a slide_number (01 to 20).
     - Create a clear and informative slide_title.
     
3. Content Guidelines:
   - Ensure a logical flow and structure throughout the presentation.
   - Include an introduction slide (slide 02) and a conclusion slide (slide 19).   

4. Common Points:
   - Include the most important key

### 전체 프레젠테이션의 Outline을 만드는 prompt 실행

In [5]:
print("\n## Generating Overall Outline by Claude 3.5 Sonnet:\n")
outline, input_tokens, output_tokens = invoke_claude(outline_prompt)

print(outline)

print("\n" + "="*50 + "\n")
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
print("\n" + "="*50 + "\n")


## Generating Overall Outline by Claude 3.5 Sonnet:

{
    "title": "2024 파리 올림픽 폐회식 성공 예측: 전망과 기대",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐회식 성공 예측: 전망과 기대"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 파리 올림픽의 의미와 폐회식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽 개요: 주요 특징 및 목표"
        },
        {
            "slide_number": "04",
            "slide_title": "역사적 맥락: 과거 올림픽 폐회식의 성공 사례"
        },
        {
            "slide_number": "05",
            "slide_title": "파리의 문화적 요소: 폐회식에 반영될 예술과 전통"
        },
        {
            "slide_number": "06",
            "slide_title": "기술 혁신: 폐회식에 도입될 첨단 기술"
        },
        {
            "slide_number": "07",
            "slide_title": "지속 가능성: 환경 친화적 폐회식 계획"
        },
        {
            "slide_number": "08",
            "slide_title": "글로벌 참여: 전 세계 시청자 유치 전략"
        },
       

In [6]:
# Outline에서 슬라이드 타이틀만 보기
print_slide_titles(outline)

프레젠테이션 제목: 2024 파리 올림픽 폐회식 성공 예측: 전망과 기대

슬라이드 01: 2024 파리 올림픽 폐회식 성공 예측: 전망과 기대
슬라이드 02: 소개: 파리 올림픽의 의미와 폐회식의 중요성
슬라이드 03: 파리 2024 올림픽 개요: 주요 특징 및 목표
슬라이드 04: 역사적 맥락: 과거 올림픽 폐회식의 성공 사례
슬라이드 05: 파리의 문화적 요소: 폐회식에 반영될 예술과 전통
슬라이드 06: 기술 혁신: 폐회식에 도입될 첨단 기술
슬라이드 07: 지속 가능성: 환경 친화적 폐회식 계획
슬라이드 08: 글로벌 참여: 전 세계 시청자 유치 전략
슬라이드 09: 안전 및 보안: 폐회식 중 관중 보호 대책
슬라이드 10: 문화적 다양성: 국제적 요소의 통합
슬라이드 11: 경제적 영향: 폐회식의 경제적 파급 효과
슬라이드 12: 미디어 전략: 글로벌 방송 및 디지털 플랫폼 활용
슬라이드 13: 참여형 요소: 관중과 선수들의 상호작용 계획
슬라이드 14: 음악과 공연: 세계적 아티스트의 참여
슬라이드 15: 상징적 의미: 올림픽 정신의 표현
슬라이드 16: 로지스틱스: 대규모 행사 관리 및 조정
슬라이드 17: 레거시: 폐회식 이후의 장기적 영향
슬라이드 18: 도전과 위험: 잠재적 문제점과 대응 방안
슬라이드 19: 결론: 성공적인 폐회식을 위한 핵심 요소
슬라이드 20: 감사합니다 & Q&A


### Slide prompt template 로딩

In [7]:
# Slide Prompt template
prompt_file_name = "slide_prompt_template.txt"

slide_prompt_file_path = os.path.join(prompt_folder, prompt_file_name)

template_content = load_template_from_file(slide_prompt_file_path)
slide_prompt_template = Template(template_content)
slide_prompt = slide_prompt_template.render()

print("\n## Slide prompt template...")
print(slide_prompt)



## Slide prompt template...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this  of the , Generate detailed content for slides  to .
For each slide, include:
1. Slide title
2. Main content points (3-5 bullet points)
3. Relevant data or examples
4. Suggesion for visuals (e.g., image, chart, graph)
5. A brief speaker note (1-2 sentences)


### 만들어진 슬라이드를 저장할 결과 폴더 생성

In [16]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

### Slide prompt template을 기반으로 슬라이드 1 - 10 번을 생성할 Slide prompt 생성

In [17]:
from util.cross_region_inference_prompt_util import create_slide_prompt, generate_slide_content
# topic = "Paris 2024 Olympics Closing Ceremony Success Prediction"

print("\n## Generating prompt for creating the content of slides 1-10...")
slide_prompt = create_slide_prompt(slide_prompt_template, TOPIC, outline, 1, 10)
print(slide_prompt)

# 1 - 10번째 슬라이드 생성
print("\n## Generating content for slides 1-10...")
include_outline = False
slides_1_10 = generate_slide_content(slide_prompt, outline, output_directory, include_outline, 1, 10, 1)


## Generating prompt for creating the content of slides 1-10...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this {
    "title": "2024 파리 올림픽 폐회식 성공 예측: 전망과 기대",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐회식 성공 예측: 전망과 기대"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 파리 올림픽의 의미와 폐회식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽 개요: 주요 특징 및 목표"
        },
        {
            "slide_number": "04",
            "slide_title": "역사적 맥락: 과거 올림픽 폐회식의 성공 사례"
        },
        {
            "slide_number": "05",
            "slide_title": "파리의 문화적 요소: 폐회식에 반영될 예술과 전통"
        },
        {
            "slide_number": "06",
           

### Slide prompt template을 기반으로 슬라이드 11 - 20 번을 생성할 Slide prompt 생성

In [18]:
from util.v2_prompt_util import create_slide_prompt, generate_slide_content
# topic = "Paris 2024 Olympics Closing Ceremony Success Prediction"

print("\nGenerating prompt for creating the content of slides 11-20...")
slide_prompt = create_slide_prompt(slide_prompt_template, TOPIC, outline, 11, 20)
print(slide_prompt)

# 11 - 20번째 슬라이드 생성
print("\nGenerating content for slides 11-20...")
include_outline = False
slides_1_10 = generate_slide_content(slide_prompt, outline, output_directory, include_outline, 11, 20, 2)


Generating prompt for creating the content of slides 11-20...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this {
    "title": "2024 파리 올림픽 폐회식 성공 예측: 전망과 기대",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐회식 성공 예측: 전망과 기대"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 파리 올림픽의 의미와 폐회식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽 개요: 주요 특징 및 목표"
        },
        {
            "slide_number": "04",
            "slide_title": "역사적 맥락: 과거 올림픽 폐회식의 성공 사례"
        },
        {
            "slide_number": "05",
            "slide_title": "파리의 문화적 요소: 폐회식에 반영될 예술과 전통"
        },
        {
            "slide_number": "06",
            "


 Detailed Slide Contents:

--------------------------------------------------

11. 경제적 영향: 폐회식의 경제적 파급 효과

• 주요 내용:
- 관광객 유치와 지역 경제 활성화
- 파리 브랜드 가치 상승 효과
- 장기적 투자 유치 및 일자리 창출
- 올림픽 관련 산업의 성장

• 데이터/예시:
- 2012 런던 올림픽: 약 95억 파운드의 경제적 이익 창출
- 2020 도쿄 올림픽: 약 2조 엔의 경제 효과 추정

• 시각화 제안: 올림픽 전후 파리 경제 지표 변화를 보여주는 그래프

• 발표자 노트: 폐회식은 단순한 행사를 넘어 파리와 프랑스 경제에 큰 영향을 미칠 것입니다. 장기적인 경제적 이익을 극대화하기 위한 전략이 중요합니다.

12. 미디어 전략: 글로벌 방송 및 디지털 플랫폼 활용

• 주요 내용:
- 실시간 글로벌 방송 네트워크 구축
- 소셜 미디어 플랫폼을 통한 실시간 상호작용
- 가상현실(VR) 및 증강현실(AR) 기술 도입
- 멀티 스크린 경험 제공

• 데이터/예시:
- 2021 도쿄 올림픽: 전 세계 30억 명 이상 시청
- 소셜 미디어 참여도: 인스타그램 팔로워 500만 명 이상

• 시각화 제안: 다양한 미디어 플랫폼을 통한 글로벌 시청자 분포도

• 발표자 노트: 디지털 시대에 맞는 혁신적인 미디어 전략으로 전 세계 시청자들에게 몰입감 있는 폐회식 경험을 제공할 것입니다.

13. 참여형 요소: 관중과 선수들의 상호작용 계획

• 주요 내용:
- 관중 참여형 공연 및 이벤트
- 선수들과 관중의 실시간 소통 기회
- 디지털 플랫폼을 통한 글로벌 팬 참여
- 문화적 교류 촉진 프로그램

• 데이터/예시:
- 현장 관중 8만 명 참여 목표
- 글로벌 온라인 참여자 1억 명 예상

• 시각화 제안: 관중 참여 활동을 보여주는 인포그래픽

• 발표자 노트: 관중과 선수들의 적극적인 참여는 폐회식을 더욱 의미 있고 기억에 남는 행사로 만들 것입니다.

14. 음악과 공연: 세

### 두 차례 호출로 만들어졌던 슬라이드 내용을 한 파일에 최종적으로 모으기

In [19]:
# Merge results
merge_files(output_directory)

All files have been merged into 'final_result.txt' in the directory './results/2024-08-29-00-58-19'.


## 2: Zero shot prompting 이지만 Role-based prompting, Task-specific instructions, Content structuring, Output constraints를 사용해서 파워포인트를 생성하기
: Structure the presentation을 미리 prompt (presentation_prompt) 에 써주고, 각 슬라이드의 아웃풋 토큰 개수에 제한을 두기

### 프롬프트 로딩

In [12]:
from jinja2 import Template

all_prompt_file_name = "allinall_prompt_template.txt"

prompt_file_path = os.path.join(prompt_folder, all_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Closing Ceremony Success Prediction"
presentation_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)
# print_json(prompt)
print(presentation_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, 
ensuring a logical flow and comprehensive coverage of the topic.

<topic>
Paris 2024 Olympics Closing Ceremony Success Prediction
</topic>

<instruction>
For each slide, provide:
1. A clear and concise title
2. 3-5 bullet points of key information
3. Any relevant data or statistics (if applicable)

Structure the presentation as follows:
- Slide 1: Introduction
- Slides 2-3: Background and Context
- Slides 4-7: Main Points and Analysis
- Slides 8-9: Predictions and Potential Outcomes
- Slide 10: Conclusion and Key Takeaways

Present the content for all 20 slides in a clear, organized manner. 
The number of output tokens for each slide must not exceed 100
</instruction>


### 생성한 슬라이드 결과를 저장할 폴더 생성

In [13]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

### 슬라이드 생성

In [15]:
from util.cross_region_inference_prompt_util import generate_all_slide_content

generate_all_slide_content(presentation_prompt, output_directory)

Generated Presentation Content:
Here's a detailed outline for a 20-slide presentation on "Paris 2024 Olympics Closing Ceremony Success Prediction" in Korean:

1. 제목: 파리 2024 올림픽 폐회식 성공 예측
• 발표 주제 소개
• 발표자 소개
• 발표 개요

2. 파리 2024 올림픽 개요
• 개최 일정: 2024년 7월 26일 - 8월 11일
• 주요 경기장 소개
• 참가국 및 선수 예상 수

3. 올림픽 폐회식의 중요성
• 올림픽 정신 전달
• 개최국 문화 홍보
• 전 세계 시청자 유치

4. 파리의 올림픽 개최 경험
• 1900년 올림픽 개최
• 1924년 올림픽 개최
• 과거 경험이 2024년에 미칠 영향

5. 파리의 문화적 자산
• 에펠탑, 루브르 등 랜드마크 활용
• 프랑스 예술과 음악의 통합
• 다문화 요소 반영

6. 기술 혁신과 폐회식
• AR/VR 기술 활용 가능성
• 드론 쇼 예상
• 친환경 기술 도입

7. 안전 및 보안 대책
• 테러 위협 대응 방안
• COVID-19 대응 계획
• 관중 관리 전략

8. 예산 및 경제적 영향
• 폐회식 예산 추정: €약 1억
• 경제적 파급 효과 예측
• 스폰서십 현황

9. 시청률 예측
• 과거 올림픽 폐회식 시청률 비교
• 디지털 플랫폼 시청자 수 예상
• 글로벌 시간대별 시청률 예측

10. 결론 및 주요 시사점
• 성공 가능성 높음
• 문화와 기술의 조화가 관건
• 글로벌 화합의 메시지 전달 중요

11. 폐회식 주제 예측
• "미래를 향한 비전" 가능성
• 지속가능성 강조 예상
• 세계 평화 메시지 포함

12. 주요 출연진 예상
• 프랑스 유명 아티스트 참여
• 국제적 셀러브리티 초청
• 올림픽 영웅들의 특별 출연

13. 음악 및 공연 요소
• 클래식과 현대음악의 조화
• 프랑스 전통 춤 포함
• 다국적 아티스트 협연 가능성

14. 시각적 효과 및 연출
• 첨